# Copia ordenada - Novo Giro Principalidade (debug)
Objetivo: notebook limpio y ordenado para ejecutar el flujo de cálculo de caídas de principalidade,
derivación de productos faltantes y análisis de caída de uso de tarjeta interno. Ejecutar celdas en orden.

In [1]:
# Imports y setup inicial (celda 1: entorno)
%cd "C:\Git\BI0730"
import os, sys, re, glob
import duckdb
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
sys.path.append('../../')
sys.path.append(rf"C:\Git\BI0730")
from libs.geral.utils import *
from libs.geral.df_pipes import *
from libs.geral.myconstants import *
from libs.denodo.conexaoDenodo import conexao_denodo as cnn
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None
user = os.getlogin()

C:\Git\BI0730


In [2]:
# Rutas y constantes (celda 2)
BASE_DIR = rf"C:\Users\{user}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira"
PRINCIPALIDADE_PATH = os.path.join(BASE_DIR, "indicador_nova_principalidade_historico.parquet")
EMI_CARTOES_PATH = os.path.join(BASE_DIR, "emi_compras_confirmacao_historico.parquet")
PATH_BASES_PARQUET = globals().get('PATH_BASES_PARQUET', rf"C:\Users\{user}\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet")
RUTAS = {
    'inadimplentes': os.path.join(PATH_BASES_PARQUET, 'base_inadimplentes.parquet'),
    'associados_dir': os.path.join(PATH_BASES_PARQUET, 'associados_total_diario'),
    'saida_dir': BASE_DIR,
    'saida_dir_giro': os.path.join(BASE_DIR, 'giro_de_carteira'),
    'principalidade': PRINCIPALIDADE_PATH,
    'cartoes':os.path.join(PATH_BASES_PARQUET,'emi_compras_confirmacao_historico')
}
os.makedirs(RUTAS['saida_dir_giro'], exist_ok=True)
ANO = date.today().year
data = date.today().strftime('%Y%m')
PARAMS = {
    'filtros_risco_bbm': ["BAIXÍSSIMO", "BAIXO 1", "BAIXO 2", "MÉDIO 1", "MÉDIO 2"],
    'dias_sem_movimentacao': (20, 45)
}

In [3]:
# DuckDB: crear tabla 'dados' desde parquet y calcular deltas/flags (celda 3)
con = duckdb.connect()

if not os.path.exists(PRINCIPALIDADE_PATH):
    raise FileNotFoundError(PRINCIPALIDADE_PATH)

con.execute(f"CREATE OR REPLACE TABLE dados AS SELECT * FROM read_parquet('{PRINCIPALIDADE_PATH.replace('\\\\','\\\\\\\\')}')")

query = r'''
WITH tratados AS (
  SELECT *,
    ROW_NUMBER() OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes DESC) AS rn,
    pontos_principalidade - LAG(pontos_principalidade) OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes ASC) AS var_pontos,
    COALESCE(possui_cartao_credito,0) - COALESCE(LAG(possui_cartao_credito) OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes ASC),0) AS delta_cartao,
    COALESCE(cad_pix_ativo,0) - COALESCE(LAG(cad_pix_ativo) OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes ASC),0) AS delta_pix,
    COALESCE(debito_conta_ativo,0) - COALESCE(LAG(debito_conta_ativo) OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes ASC),0) AS delta_debito,
    COALESCE(possui_cartao_debito,0) - COALESCE(LAG(possui_cartao_debito) OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes ASC),0) AS delta_cartao_debito,
    COALESCE(transacao_app,0) - COALESCE(LAG(transacao_app) OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes ASC),0) AS delta_transacao_app,
    COALESCE(ativou_open_finance,0) - COALESCE(LAG(ativou_open_finance) OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes ASC),0) AS delta_open_finance,
    COALESCE(possui_folha_pagamento,0) - COALESCE(LAG(possui_folha_pagamento) OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes ASC),0) AS delta_folha,
    COALESCE(possui_cobranca,0) - COALESCE(LAG(possui_cobranca) OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes ASC),0) AS delta_cobranca,
    COALESCE(possui_domicilio,0) - COALESCE(LAG(possui_domicilio) OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes ASC),0) AS delta_domicilio,
    COALESCE(possui_adquirencia,0) - COALESCE(LAG(possui_adquirencia) OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes ASC),0) AS delta_adquirencia
  FROM dados
  QUALIFY rn <= 4
  ORDER BY ano_mes
),
marcacoes AS (
  SELECT *,
    CASE WHEN var_pontos < 0 THEN 1 ELSE 0 END AS queda_flag,
    CASE WHEN delta_cartao < 0 THEN 1 ELSE 0 END AS lost_cartao_flag,
    CASE WHEN delta_cartao > 0 THEN 1 ELSE 0 END AS gained_cartao_flag,
    CASE WHEN delta_pix < 0 THEN 1 ELSE 0 END AS lost_pix_flag,
    CASE WHEN delta_pix > 0 THEN 1 ELSE 0 END AS gained_pix_flag,
    CASE WHEN delta_debito < 0 THEN 1 ELSE 0 END AS lost_debito_flag,
    CASE WHEN delta_debito > 0 THEN 1 ELSE 0 END AS gained_debito_flag,
    CASE WHEN delta_cartao_debito < 0 THEN 1 ELSE 0 END AS lost_cartao_debito_flag,
    CASE WHEN delta_cartao_debito > 0 THEN 1 ELSE 0 END AS gained_cartao_debito_flag,
    CASE WHEN delta_transacao_app < 0 THEN 1 ELSE 0 END AS lost_transacao_app_flag,
    CASE WHEN delta_transacao_app > 0 THEN 1 ELSE 0 END AS gained_transacao_app_flag,
    CASE WHEN delta_open_finance < 0 THEN 1 ELSE 0 END AS lost_open_flag,
    CASE WHEN delta_open_finance > 0 THEN 1 ELSE 0 END AS gained_open_flag,
    CASE WHEN delta_folha < 0 THEN 1 ELSE 0 END AS lost_folha_flag,
    CASE WHEN delta_folha > 0 THEN 1 ELSE 0 END AS gained_folha_flag,
    CASE WHEN delta_cobranca < 0 THEN 1 ELSE 0 END AS lost_cobranca_flag,
    CASE WHEN delta_cobranca > 0 THEN 1 ELSE 0 END AS gained_cobranca_flag,
    CASE WHEN delta_domicilio < 0 THEN 1 ELSE 0 END AS lost_domicilio_flag,
    CASE WHEN delta_domicilio > 0 THEN 1 ELSE 0 END AS gained_domicilio_flag,
    CASE WHEN delta_adquirencia < 0 THEN 1 ELSE 0 END AS lost_adquirencia_flag,
    CASE WHEN delta_adquirencia > 0 THEN 1 ELSE 0 END AS gained_adquirencia_flag
  FROM tratados
  WHERE rn <= 3
),
checagem AS (
  SELECT *,
    SUM(queda_flag) OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes) AS soma_quedas,
    SUM(lost_cartao_flag) OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes) AS soma_lost_cartao,
    SUM(gained_cartao_flag) OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes) AS soma_gained_cartao,
    SUM(lost_pix_flag + lost_debito_flag + lost_cartao_debito_flag + lost_transacao_app_flag + lost_open_flag + lost_folha_flag + lost_cobranca_flag + lost_domicilio_flag + lost_adquirencia_flag)
      OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes) AS soma_lost_produtos,
    SUM(gained_pix_flag + gained_debito_flag + gained_cartao_debito_flag + gained_transacao_app_flag + gained_open_flag + gained_folha_flag + gained_cobranca_flag + gained_domicilio_flag + gained_adquirencia_flag)
      OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes) AS soma_gained_produtos
  FROM marcacoes
)
SELECT * FROM checagem
WHERE soma_quedas >= 3
  AND ano_mes = (SELECT MAX(ano_mes) FROM checagem);
'''
df_quedas = con.execute(query).df()
# materializar parquet (atomic)
saida = os.path.join(RUTAS['saida_dir_giro'], 'associados_queda_principalidade.parquet')
tmp = saida + ".tmp"
df_quedas.to_parquet(tmp, index=False)
os.replace(tmp, saida)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
# Colunas arquivo principalidade
colunas_principalidade = ['ano_mes', 'cpf_cnpj', 'segmento', 'cod_cooperativa', 'cod_central',
       'cod_agencia', 'cod_carteira', 'status_associado', 'porte_padrao',
       'meses_desde_associacao', 'assoc_desde', 'nivel_risco', 
       'vlr_capital_social',  'pix_trans_30d', 'cad_pix',
       'sum_titulo_investimento', 'mobi_transacionou_30d', 'sum_previdencia',
       'sum_investimentos', 'total_valor_cartao_mercado_scr',
       'total_valor_cartao_sicredi', 'total_valor_outros_sicredi',  'sow_cartao',
       'possui_cartao_credito', 'debito_conta_ativo', 'possui_domicilio',
       'possui_cartao_debito', 'possui_folha_pagamento', 'possui_cobranca',
       'possui_adquirencia', 'transacao_app', 'flg_ativo', 'flg_novo_assoc',  'cad_pix_ativo',
       'ativou_open_finance', 'cash_in', 'cash_out', 'cash_total_fator',
       'cash_total', 'pontos_produtos_basicos',  'flag_principalidade',
       'pontos_principalidade', 'faixa_atingimento_meta',
       'percentual_atingimento',  'faixa_categoria',  'var_pontos',
       'queda_flag', 'soma_quedas']
colunas_principalidade

['ano_mes',
 'cpf_cnpj',
 'segmento',
 'cod_cooperativa',
 'cod_central',
 'cod_agencia',
 'cod_carteira',
 'status_associado',
 'porte_padrao',
 'meses_desde_associacao',
 'assoc_desde',
 'nivel_risco',
 'vlr_capital_social',
 'pix_trans_30d',
 'cad_pix',
 'sum_titulo_investimento',
 'mobi_transacionou_30d',
 'sum_previdencia',
 'sum_investimentos',
 'total_valor_cartao_mercado_scr',
 'total_valor_cartao_sicredi',
 'total_valor_outros_sicredi',
 'sow_cartao',
 'possui_cartao_credito',
 'debito_conta_ativo',
 'possui_domicilio',
 'possui_cartao_debito',
 'possui_folha_pagamento',
 'possui_cobranca',
 'possui_adquirencia',
 'transacao_app',
 'flg_ativo',
 'flg_novo_assoc',
 'cad_pix_ativo',
 'ativou_open_finance',
 'cash_in',
 'cash_out',
 'cash_total_fator',
 'cash_total',
 'pontos_produtos_basicos',
 'flag_principalidade',
 'pontos_principalidade',
 'faixa_atingimento_meta',
 'percentual_atingimento',
 'faixa_categoria',
 'var_pontos',
 'queda_flag',
 'soma_quedas']

In [ ]:
# Carga y preparación inicial (celda 4) - asegurar esquema EXACTO según colunas_principalidade
base_path = os.path.join(RUTAS['saida_dir_giro'], 'associados_queda_principalidade.parquet')
if not os.path.exists(base_path):
    raise FileNotFoundError(base_path)
_tmp = pd.read_parquet(base_path)

# Asegurar presencia de todas las columnas listadas en colunas_principalidade
for c in colunas_principalidade:
    if c not in _tmp.columns:
        _tmp[c] = np.nan
# Añadir 'soma_lost_produtos' si no está en la lista pero sí en el dataframe
if 'soma_lost_produtos' in _tmp.columns and 'soma_lost_produtos' not in colunas_principalidade:
    colunas_principalidade.append('soma_lost_produtos')

# Seleccionar y fijar orden según colunas_principalidade
base_princ = _tmp[[c for c in colunas_principalidade if c in _tmp.columns]].copy()
# Asegurar columnas mínimas adicionales esperadas por el flujo (incluir cálculo automático de produtos_basicos_faltantes desde columnas booleanas)
prod_basic_cols = ['cad_pix_ativo','debito_conta_ativo','possui_cartao_debito','possui_cartao_credito','possui_folha_pagamento','possui_cobranca','possui_domicilio','possui_adquirencia','transacao_app','ativou_open_finance']
for c in prod_basic_cols:
    if c not in base_princ.columns:
        base_princ[c] = 0  # asumir ausente si no existe

# Calcular produtos_basicos_faltantes automáticamente desde datos reales
def _calc_faltantes_real(row):
    falt = []
    for c in prod_basic_cols:
        if c in row.index:
            v = row.get(c, 0)
            try:
                if pd.isna(v) or float(v) == 0:
                    falt.append(c)
            except Exception:
                if not v:
                    falt.append(c)
    return falt

base_princ['produtos_basicos_faltantes'] = base_princ.apply(_calc_faltantes_real, axis=1)

# Aplicar filtros: BBM y sin inadimplente
# Filtrar BBM
base_princ = base_princ[base_princ['nivel_risco'].isin(PARAMS['filtros_risco_bbm'])]

# Filtrar sin inadimplente: merge con base_inadimplentes
inad_path = RUTAS['inadimplentes']
if os.path.exists(inad_path):
    inad_df = pd.read_parquet(inad_path)
    inad_df['cpf_cnpj'] = inad_df['cpf_cnpj'].astype(str).str.replace(r'\D','', regex=True).fillna('')
    inad_df['cpf_cnpj'] = np.where(inad_df['cpf_cnpj'].str.len() > 11,
                                  inad_df['cpf_cnpj'].str.zfill(14),
                                  inad_df['cpf_cnpj'].str.zfill(11))
    base_princ = base_princ.merge(inad_df[['cpf_cnpj']], on='cpf_cnpj', how='left', indicator=True)
    base_princ = base_princ[base_princ['_merge'] == 'left_only'].drop('_merge', axis=1)

# Guardar parquet filtrado después de tratamientos
base_path = os.path.join(RUTAS['saida_dir_giro'], 'associados_queda_principalidade.parquet')
tmp = base_path + ".tmp"
base_princ.to_parquet(tmp, index=False)
os.replace(tmp, base_path)

In [6]:
princ_columns_insights = pd.read_parquet(rf"C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\giro_de_carteira\associados_queda_principalidade.parquet")

# Colunas arquivo principalidade
colunas_principalidade = ['ano_mes', 'cpf_cnpj', 'segmento', 'cod_agencia', 'cod_carteira', 'status_associado', 'porte_padrao',
       'meses_desde_associacao', 'assoc_desde', 'nivel_risco', 
       'vlr_capital_social',  'pix_trans_30d', 'cad_pix',
       'sum_titulo_investimento', 'mobi_transacionou_30d', 'sum_previdencia',
       'sum_investimentos', 'total_valor_cartao_mercado_scr',
       'total_valor_cartao_sicredi', 'total_valor_outros_sicredi',  'sow_cartao',
       'possui_cartao_credito', 'debito_conta_ativo', 'possui_domicilio',
       'possui_cartao_debito', 'possui_folha_pagamento', 'possui_cobranca',
       'possui_adquirencia', 'transacao_app', 'flg_ativo', 'flg_novo_assoc',  'cad_pix_ativo',
       'ativou_open_finance', 'cash_in', 'cash_out', 'cash_total_fator',
       'cash_total', 'pontos_produtos_basicos',  'flag_principalidade',
       'pontos_principalidade', 'faixa_atingimento_meta',
       'percentual_atingimento',  'faixa_categoria',  'var_pontos',
       'queda_flag', 'soma_quedas']
colunas_principalidade

In [7]:
princ_columns_insights

,ano_mes,cpf_cnpj,segmento,cod_cooperativa,cod_central,cod_agencia,cod_carteira,status_associado,porte_padrao,meses_desde_associacao,assoc_desde,nivel_risco,vlr_capital_social,pix_trans_30d,cad_pix,sum_titulo_investimento,mobi_transacionou_30d,sum_previdencia,sum_investimentos,total_valor_cartao_mercado_scr,total_valor_cartao_sicredi,total_valor_outros_sicredi,sow_cartao,possui_cartao_credito,debito_conta_ativo,possui_domicilio,possui_cartao_debito,possui_folha_pagamento,possui_cobranca,possui_adquirencia,transacao_app,flg_ativo,flg_novo_assoc,cad_pix_ativo,ativou_open_finance,cash_in,cash_out,cash_total_fator,cash_total,pontos_produtos_basicos,flag_principalidade,pontos_principalidade,faixa_atingimento_meta,percentual_atingimento,faixa_categoria,var_pontos,queda_flag,soma_quedas,produtos_basicos_faltantes
0,202507,00597674990,PF,0730,7000,04,142,ATIVO,PF IV,18,202401,BAIXÍSSIMO,"2,133.86",N,S,"87,563.40",S,0.00,"89,697.26",250.14,250.14,0.00,1.00,1,1,0,0,0,0,0,1,1,0,0,0,0.00,355.79,0.00,355.79,12,0,107,80-100%,82.3076923076923,Promissor,-1,1,3.00,"[cad_pix_ativo, possui_cartao_debito, possui_folha_pagamento, possui_cobranca, possui_domicilio, possui_adquirencia, ativou_open_finance]"
1,202507,01810115906,PF,0730,7000,10,121,ATIVO,PF III,55,202012,BAIXO 2,630.75,S,S,2.78,S,0.00,633.53,"14,303.58","14,303.58","39,512.79",1.00,1,0,0,1,0,0,0,1,1,0,1,0,"3,810.00","7,353.90",0.00,"11,163.90",20,1,123,120+,123.0,Maduro,-37,1,3.00,"[debito_conta_ativo, possui_folha_pagamento, possui_cobranca, possui_domicilio, possui_adquirencia, ativou_open_finance]"
2,202507,01822941962,PF,0730,7000,30,010101,ATIVO,PF III,3,202504,MÉDIO 2,5.00,N,S,0.00,N,0.00,5.00,"23,980.97",0.00,"23,522.54",0.00,0,0,0,0,0,0,0,0,1,1,0,0,0.00,0.00,0.00,0.00,0,0,8,0-20%,8.0,Sem classificação,-3,1,3.00,"[cad_pix_ativo, debito_conta_ativo, possui_cartao_debito, possui_cartao_credito, possui_folha_pagamento, possui_cobranca, possui_domicilio, possui_adquirencia, transacao_app, ativou_open_finance]"
3,202507,01863627901,PF,0730,7000,12,010101,ATIVO,PF I,38,202205,BAIXO 2,725.09,N,N,0.00,N,0.00,725.09,0.00,0.00,0.46,0.00,0,0,0,0,0,0,0,0,1,0,0,0,"1,534.00","1,524.90",0.00,"3,058.90",0,0,24,20-80%,48.0,Emergente,-1,1,3.00,"[cad_pix_ativo, debito_conta_ativo, possui_cartao_debito, possui_cartao_credito, possui_folha_pagamento, possui_cobranca, possui_domicilio, possui_adquirencia, transacao_app, ativou_open_finance]"
4,202507,01868854230,PF,0730,7000,28,122,ATIVO,PF II,25,202306,BAIXO 1,647.02,S,S,"4,104.93",S,0.00,"4,751.95","4,112.02","1,972.52","18,699.41",0.48,1,1,0,1,0,0,0,1,1,0,1,0,"4,211.00","3,971.33",0.00,"8,182.33",30,1,114,120+,142.5,Maduro,-3,1,3.00,"[possui_folha_pagamento, possui_cobranca, possui_domicilio, possui_adquirencia, ativou_open_finance]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3702,202507,07941700999,PF,0730,7000,04,010102,ATIVO,PF I,13,202406,MÉDIO 1,8.09,N,S,0.00,N,0.00,8.09,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,0,1,0,0,0,550.00,0.00,0.00,550.00,0,0,3,0-20%,6.0,Aspirante,-7,1,3.00,"[cad_pix_ativo, debito_conta_ativo, possui_cartao_debito, possui_cartao_credito, possui_folha_pagamento, possui_cobranca, possui_domicilio, possui_adquirencia, transacao_app, ativou_open_finance]"
3703,202507,07994521908,AGRO,0730,7000,03,213,ATIVO,Familiar,184,201003,BAIXÍSSIMO,"2,786.77",S,S,909.49,S,0.00,"3,696.26",0.00,0.00,"102,881.13",0.00,0,0,0,1,0,0,0,1,1,0,1,0,20.00,"7,672.52","8,345.90","7,692.52",12,0,50,80-100%,83.33333333333334,Promissor,-7,1,3.00,"[debito_conta_ativo, possui_cartao_credito, possui_folha_pagamento, possui_cobranca, possui_domicilio, possui_adquirencia, ativou_open_finance]"
3704,202507,08231492933,PF,0730,7000,29,111,ATIVO,PF III,67,201912,BAIXO 2,"1,696.02",S,S,0.78,S,0.00,"1,696.80","2,657.25",0.00,0.00,0.00,0,1,0,0,0,0,0,1,1,0,1,0,0.00,398.34,0.00,398.34,12,0,15,0-20%,15.0,Aspirante,-

In [8]:
# Carga dashboard e ISA histórico (celda 6) - conservar columnas de contacto (comercial, celular, telefone)
dash_path = os.path.join(PATH_BASES_PARQUET, 'associados_totais_tratados_dashboard.parquet')
if os.path.exists(dash_path):
    base_cols_dash = ['cpf_cnpj','num_conta','nom_associado','mc_total_1','gestor_ldap','telefone','celular','comercial','gestor']
    try:
        assoc_dash = pd.read_parquet(dash_path, columns=base_cols_dash + ['dt_ult_movimento'])
    except Exception:
        try:
            assoc_dash = pd.read_parquet(dash_path, columns=base_cols_dash)
        except Exception:
            assoc_dash = pd.DataFrame(columns=base_cols_dash + ['ult_movimento'])
    # normalizar nombre de fecha si existe
    if 'dt_ult_movimento' in assoc_dash.columns and 'ult_movimento' not in assoc_dash.columns:
        assoc_dash = assoc_dash.rename(columns={'dt_ult_movimento':'ult_movimento'})
    if 'ult_movimento' not in assoc_dash.columns:
        assoc_dash['ult_movimento'] = pd.NaT

    # Normalización consistente para llaves (sin tocar columnas de contacto)
    assoc_dash['cpf_cnpj'] = assoc_dash['cpf_cnpj'].astype(str).str.replace(r'\D','', regex=True).fillna('')
    assoc_dash['cpf_cnpj'] = np.where(assoc_dash['cpf_cnpj'].str.len() > 11,
                                      assoc_dash['cpf_cnpj'].str.zfill(14),
                                      assoc_dash['cpf_cnpj'].str.zfill(11))
    if 'num_conta' in assoc_dash.columns:
        assoc_dash['num_conta'] = assoc_dash['num_conta'].astype(str).fillna('').str.zfill(6)
    if 'cod_agencia' in assoc_dash.columns:
        assoc_dash['cod_agencia'] = assoc_dash['cod_agencia'].astype(str).fillna('').str.zfill(2)
    if 'cod_carteira' in assoc_dash.columns:
        assoc_dash['cod_carteira'] = assoc_dash['cod_carteira'].astype(str).fillna('')
        assoc_dash['cod_carteira'] = np.where(assoc_dash['cod_carteira'].str.len() > 3,
                                              assoc_dash['cod_carteira'].str.zfill(6),
                                              assoc_dash['cod_carteira'].str.zfill(3))
else:
    assoc_dash = pd.DataFrame(columns=['cpf_cnpj','num_conta','nom_associado','mc_total_1','gestor_ldap','telefone','celular','comercial','ult_movimento','gestor'])

In [9]:
# Construir el dict "descricao" (Acesso + Fluxo de Caixa) desde la fila seleccionada y guardar markdown en base_princ['descricao'] (corrección de indentación y robustez)
from IPython.display import Markdown, display
import os, pandas as pd, numpy as np

if 'base_princ' not in globals() or base_princ.empty:
    raise RuntimeError("sem resposta")

# Selección de fila objetivo: priorizar candidates_ranked si existe
if 'candidates_ranked' in globals() and isinstance(candidates_ranked, pd.DataFrame) and not candidates_ranked.empty:
    cpf_sel = str(candidates_ranked.iloc[0].get('cpf_cnpj',''))
    sel = base_princ[base_princ['cpf_cnpj'].astype(str)==cpf_sel]
    idx = sel.index[0] if not sel.empty else base_princ.index[0]
else:
    idx = base_princ.index[0]

r = base_princ.loc[idx]

# Mapeos basados en esquema (solo columnas válidas)
prod_cols_map = {

    'Pix': 'cad_pix_ativo',
    'Débito automático': 'debito_conta_ativo',
    'Conta salário': 'possui_folha_pagamento',
    'Cartão de débito': 'possui_cartao_debito',
    'Cartão de crédito': 'possui_cartao_credito',
    'Canais digitais': 'transacao_app',
    'Open Finance (receptor)': 'ativou_open_finance',
    'Domicílio': 'possui_domicilio',
    'Credenciamento ativo': 'possui_adquirencia',
    'Cobrança': 'possui_cobranca',
    'Folha de pagamento': 'possui_folha_pagamento'
}
# Filtrar a columnas presentes
prod_cols_map = {k:v for k,v in prod_cols_map.items() if v in base_princ.columns}

fluxo_map = {
    'Valor movimentado': 'cash_total',
    'Cartão (SOW)': 'sow_cartao',
    'Cash-in': 'cash_in',
    'Cash-out': 'cash_out'
}
fluxo_map = {k:v for k,v in fluxo_map.items() if v in base_princ.columns}

def _fmt(v):
    try:
        if pd.isna(v): return ''
        return f"{float(v):,.2f}"
    except Exception:
        return str(v)

# Construir Acesso (booleans) usando solo columnas válidas
acesso = {label: (bool(float(r.get(col,0)) != 0) if col in r.index else False) for label,col in prod_cols_map.items()}

# Construir Fluxo de Caixa (texto formateado)
fluxo = {}
for label, col in fluxo_map.items():
    if col in r.index and pd.notnull(r.get(col)):
        fluxo[label] = _fmt(r.get(col))

# Montar dict "descricao"
descricao_dict = {"Acesso": acesso, "Fluxo de Caixa": fluxo}

# Derivar 'nome' del associado (evitar NameError) con fallback a assoc_dash
nome = ""
# Intentar directamente en la fila (si en algún momento fue mergeado)
for nome_col in ("nom_associado", "nome_associado", "nome"):
    if nome_col in r.index and pd.notnull(r.get(nome_col)):
        nome = str(r.get(nome_col))
        break
# Fallback lookup en assoc_dash
if not nome and 'assoc_dash' in globals() and isinstance(assoc_dash, pd.DataFrame) and not assoc_dash.empty:
    _match = assoc_dash[assoc_dash['cpf_cnpj'].astype(str) == str(r.get('cpf_cnpj'))]
    if not _match.empty:
        nome = str(_match.iloc[0].get('nom_associado', "")) or nome

# Generar markdown (corrección de indentación y robustez)
def gerar_markdown(d, nome_associado):
    md = ["## DADOS ASSOCIADO"]
    md.append(f"- Nome Associado: {nome_associado}")
    md.append(f"- CPF/CNPJ: {r.get('cpf_cnpj','')}")
    md.append(f"- Agencia: {r.get('cod_agencia','')}")
    if 'num_conta' in r.index and pd.notnull(r.get('num_conta')):
        md.append(f"- Conta: {r.get('num_conta')}")
    md.append("")
    md.append("## Grupos de Soluciones")
    md.append("")
    md.append("### 🔑 Acceso")
    for k, v in d["Acesso"].items():
        simbolo = "✅" if v else "❌"
        md.append(f"- {simbolo} {k}")
    md.append("")
    md.append("### 💰 Fluxo de Caixa")
    if d["Fluxo de Caixa"]:
        for k, v in d["Fluxo de Caixa"].items():
            md.append(f"- **{k}:** {v}")
    else:
        md.append("- Sem métricas de fluxo disponiveis")
    return "\n".join(md)

markdown_simbolo = gerar_markdown(descricao_dict, nome)
base_princ.at[idx, 'descricao'] = markdown_simbolo
display(Markdown(markdown_simbolo))

## DADOS ASSOCIADO
- Nome Associado: ANDREA KLUPPEL
- CPF/CNPJ: 00597674990
- Agencia: 04

## Grupos de Soluciones

### 🔑 Acceso
- ❌ Pix
- ✅ Débito automático
- ❌ Conta salário
- ❌ Cartão de débito
- ✅ Cartão de crédito
- ✅ Canais digitais
- ❌ Open Finance (receptor)
- ❌ Domicílio
- ❌ Credenciamento ativo
- ❌ Cobrança
- ❌ Folha de pagamento

### 💰 Fluxo de Caixa
- **Valor movimentado:** 355.79
- **Cartão (SOW):** 1.00
- **Cash-in:** 0.00
- **Cash-out:** 355.79

Siguientes pasos:
- Validar nombres exactos de columnas en emi_compras_confirmacao_historico y ajustar cond_interna si la marca/tpo_produto usan otra nomenclatura.  
- Elegir 3 agencias piloto y aplicar filtro por cod_agencia.  
- Integrar campos de contacto (teléfonos/email) desde asociados_total_diario para completar "descricao".

In [25]:
# Generar markdown por fila: título, usuarioResponsavel, usuarioSolicitante y descricao (máx 10.000 chars)
from IPython.display import Markdown, display
import pandas as pd, numpy as np, os, re, json

# Helpers / fallbacks
def _fmt(v):
    try:
        if pd.isna(v): return ''
        return f"{float(v):,.2f}"
    except Exception:
        return str(v)

# fallback lookup_assoc_info si no existe en imports
def lookup_assoc_info_fallback(cpf):
    cpf_n_raw = re.sub(r'\D','', str(cpf)).lstrip('0')
    # normalizar cpf dado según regla (zfill 11/14)
    cpf_n = cpf_n_raw.zfill(14) if len(cpf_n_raw) > 11 else cpf_n_raw.zfill(11)

    res = {'nome': None, 'conta': None, 'telefones': [], 'emails': []}
    try:
        # intentar assoc_contacts (cargado en una celda anterior) o assoc_dash
        if 'assoc_contacts' in globals() and assoc_contacts is not None:
            df = assoc_contacts.copy()
            if 'cpf_cnpj' not in df.columns:
                df['cpf_cnpj'] = df.iloc[:,0].astype(str).fillna('')
            df['cpf_cnpj'] = df['cpf_cnpj'].astype(str).str.replace(r'\D','', regex=True).fillna('')
            df['cpf_cnpj'] = np.where(df['cpf_cnpj'].str.len() > 11, df['cpf_cnpj'].str.zfill(14), df['cpf_cnpj'].str.zfill(11))
            if 'num_conta' in df.columns:
                df['num_conta'] = df['num_conta'].astype(str).fillna('').str.zfill(6)
            sel = df[df['cpf_cnpj'] == cpf_n]
            if not sel.empty:
                row = sel.iloc[0]
                # nombre/conta
                for c in ['nom_associado','nome','nome_associado','nom_ros']:
                    if c in row.index and pd.notnull(row.get(c)):
                        res['nome'] = row.get(c)
                        break
                for c in ['num_conta','conta','num_conta_corrente','conta_corrente']:
                    if c in row.index and pd.notnull(row.get(c)):
                        res['conta'] = row.get(c)
                        break
                # PRIORIDAD EXPLÍCITA de campos de contacto: celular, comercial, telefone
                for phone_col in ('celular','comercial','telefone'):
                    if phone_col in df.columns:
                        v = row.get(phone_col)
                        if pd.notnull(v):
                            res['telefones'].append(str(v))
                # emails (fallback por nombres de columna)
                email_cols = [c for c in df.columns if re.search(r'email|e-mail', c, re.I)]
                for c in email_cols:
                    v = row.get(c)
                    if pd.notnull(v):
                        res['emails'].append(str(v))
    except Exception:
        pass
    # fallback assoc_dash
    try:
        if (not res['nome']) and 'assoc_dash' in globals() and not assoc_dash.empty:
            sd = assoc_dash.copy()
            # assoc_dash ya normalizado en carga; buscar por cpf normalizado
            sd_sel = sd[sd['cpf_cnpj'] == cpf_n]
            if not sd_sel.empty:
                row = sd_sel.iloc[0]
                res['nome'] = res['nome'] or row.get('nom_associado')
                res['conta'] = res['conta'] or row.get('num_conta')
                for phone_col in ('celular','comercial','telefone'):
                    if phone_col in row.index and pd.notnull(row.get(phone_col)):
                        res['telefones'].append(str(row.get(phone_col)))
                # emails
                for c in sd.columns:
                    if re.search(r'email|e-mail', c, re.I) and pd.notnull(row.get(c)):
                        res['emails'].append(str(row.get(c)))
    except Exception:
        pass
    return res

# usar lookup_assoc_info si existe, sino fallback
lookup_assoc_info = globals().get('lookup_assoc_info', lookup_assoc_info_fallback)

# Función que genera el markdown por fila
def gerar_markdown_row(r):
    segmento = r.get('segmento', '').upper()
    
    # Definições locais seguras para evitar NameError quando variáveis globais não existirem
    # mc_total: usar coluna 'cash_total' (ou 'mc_total' se existir) e já formatar
    try:
        _mc_raw = r.get('cash_total', None)
        if _mc_raw is None:
            _mc_raw = r.get('mc_total', None)
        mc_total = _fmt(_mc_raw) if _mc_raw is not None and _mc_raw != '' else ''
    except Exception:
        mc_total = ''
    
    # Variáveis ISA (se existirem nas colunas)
    isa_atual = r.get('isa_media', None)
    isa_3m = r.get('isa_media_3m_calc', None)
    
    # Pontos / variação / quedas
    pontos = r.get('pontos_principalidade', '')
    var_pontos = r.get('var_pontos', '')
    soma_quedas = r.get('soma_quedas', '')
    
    # Assoc info (fallback se lookup falhar)
    try:
        assoc_info = lookup_assoc_info(r.get('cpf_cnpj'))
    except Exception:
        assoc_info = {'nome': None, 'conta': None, 'telefones': [], 'emails': []}
    
    telefones = assoc_info.get('telefones', []) or []
    emails = assoc_info.get('emails', []) or []
    
    # Flag transacionou 30d (considerar mobi_transacionou_30d ou pix_trans_30d)
    trans_30_flag = None
    for cand in ('mobi_transacionou_30d','pix_trans_30d'):
        if cand in r.index:
            trans_30_flag = r.get(cand)
            break
    trans_30 = 'V' if (isinstance(trans_30_flag,(int,float)) and trans_30_flag != 0) or str(trans_30_flag).upper() in ('S','1','TRUE','T') else 'X'
    
    # Produtos faltantes
    faltantes = r.get('produtos_basicos_faltantes', []) or []
    
    # Perdidos (não há coluna explícita, manter vazio para evitar NameError)
    perdidos = []
    
    # Linha SOW cartão
    sow_line = ''
    if 'sow_cartao' in r.index:
        try:
            sow_val = r.get('sow_cartao')
            if pd.notnull(sow_val):
                # Se valor <=1 supõe fracionário, converte para %
                sow_pct = sow_val * 100 if sow_val <= 1 else sow_val
                cond = "menor que 60%" if sow_pct < 60 else ">= 60%"
                sow_line = f"Share of Wallet de Cartão: {int(round(sow_pct))}% ({cond})"
        except Exception:
            pass
    
    data_atualizacao = r.get('data_atualizacao', None)
    pre_aprovado = r.get('pre_aprovado', None)
    
    # Usuário / título (fallback simples se não existirem globais)
    try:
        titulo
    except NameError:
        # Montar título: usa nome (se existir) e conta
        _nome_tmp = assoc_info.get('nome') or r.get('nom_associado') or ''
        _conta_tmp = assoc_info.get('conta') or r.get('num_conta', '')
        titulo_local = f"CONTATO DO DIA: {_nome_tmp} - {_conta_tmp}".strip()
    else:
        titulo_local = titulo
    
    try:
        usuarioResponsavel
    except NameError:
        usuarioResponsavel = 'LDAP_REGIONAL'
    try:
        usuarioSolicitante
    except NameError:
        usuarioSolicitante = 'LDAP_REGIONAL'
    try:
        numeroAgencia
    except NameError:
        # converter cod_agencia se numérico
        ag = r.get('cod_agencia')
        try:
            numeroAgencia = int(str(ag))
        except Exception:
            numeroAgencia = None
    
    # Mapeos dinâmicos por segmento (produtos que NÃO tem, pois são diferentes)
    prod_maps = {
        'PF': {
            'Pix': 'cad_pix_ativo',
            'Débito automático': 'debito_conta_ativo',
            'Cartão de débito': 'possui_cartao_debito',
            'Cartão de crédito': 'possui_cartao_credito',
            'Conta salário': 'possui_folha_pagamento',
            'Canais digitais': 'transacao_app',
            'Open Finance (receptor)': 'ativou_open_finance'
        },
        'PJ': {
            'Pix': 'cad_pix_ativo',
            'Débito automático': 'debito_conta_ativo',
            'Cartão de débito': 'possui_cartao_debito',
            'Cartão de crédito': 'possui_cartao_credito',
            'Cobrança': 'possui_cobranca',
            'Domicílio': 'possui_domicilio',
            'Credenciamento ativo': 'possui_adquirencia',
            'Canais digitais': 'transacao_app',
            'Open Finance (receptor)': 'ativou_open_finance'
        },
        'AGRO': {
            'Pix': 'cad_pix_ativo',
            'Débito automático': 'debito_conta_ativo',
            'Cartão de débito': 'possui_cartao_debito',
            'Cartão de crédito': 'possui_cartao_credito',
            'Cobrança': 'possui_cobranca',
            'Domicílio': 'possui_domicilio',
            'Credenciamento ativo': 'possui_adquirencia',
            'Canais digitais': 'transacao_app',
            'Open Finance (receptor)': 'ativou_open_finance'
        }
    }
    
    prod_cols_map = prod_maps.get(segmento, prod_maps['PF'])  # default PF se não encontrado
    
    # Filtrar a columnas presentes
    prod_cols_map = {k:v for k,v in prod_cols_map.items() if v in r.index}
    
    fluxo_map = {
        'Valor movimentado': 'cash_total',
        'Cartão (SOW)': 'sow_cartao',
        'Cash-in': 'cash_in',
        'Cash-out': 'cash_out'
    }
    fluxo_map = {k:v for k,v in fluxo_map.items() if v in r.index}
    
    def _fmt(v):
        try:
            if pd.isna(v): return ''
            return f"{float(v):,.2f}"
        except Exception:
            return str(v)
    
    # Construir Acesso (booleans) usando solo columnas válidas
    acesso = {label: (bool(float(r.get(col,0)) != 0) if col in r.index else False) for label,col in prod_cols_map.items()}
    
    # Construir Fluxo de Caixa (texto formateado)
    fluxo = {}
    for label, col in fluxo_map.items():
        if col in r.index and pd.notnull(r.get(col)):
            fluxo[label] = _fmt(r.get(col))
    
    # Montar dict "descricao"
    descricao_dict = {"Acesso": acesso, "Fluxo de Caixa": fluxo}
    
    # Derivar 'nome' del associado
    nome = assoc_info.get('nome') or r.get('nom_associado') or r.get('nome') or ''
    
    # Generar markdown (corrección de indentación y robustez)
    def gerar_markdown(d, nome_associado):
        md = ["## DADOS ASSOCIADO"]
        md.append(f"- Nome Associado: {nome_associado}")
        md.append(f"- CPF/CNPJ: {r.get('cpf_cnpj','')}")
        md.append(f"- SEGMENTO: {r.get('segmento','')}")
        md.append(f"- Agencia: {r.get('cod_agencia','')}")
        if 'num_conta' in r.index and pd.notnull(r.get('num_conta')):
            md.append(f"- Conta: {r.get('num_conta')}")
        md.append("")
        md.append("## Grupos de Soluciones")
        md.append("")
        md.append("### 🔑 Acceso")
        for k, v in d["Acesso"].items():
            simbolo = "✅" if v else "❌"
            md.append(f"- {simbolo} {k}")
        md.append("")
        md.append("### 💰 Fluxo de Caixa")
        if d["Fluxo de Caixa"]:
            for k, v in d["Fluxo de Caixa"].items():
                md.append(f"- **{k}:** {v}")
        else:
            md.append("- Sem métricas de fluxo disponiveis")
        return "\n".join(md)

    markdown_texto = gerar_markdown(descricao_dict, nome)
    
    # Ahora agregar las otras secciones
    full_md = []
    full_md.append(f"- Segmento: {r.get('segmento','')}")
    if isa_atual: full_md.append(f"- ISA atual: {_fmt(isa_atual)}")
    if isa_3m: full_md.append(f"- ISA média (3M): {_fmt(isa_3m)}")
    if mc_total: full_md.append(f"- MC Total Atual: {mc_total}")
    full_md.append(f"- Puntos principalidade: {pontos} | Var: {var_pontos}")
    if soma_quedas: full_md.append(f"- Soma quedas: {int(soma_quedas)}")
    full_md.append(f"- Percentual Atingimento: {r.get('percentual_atingimento','')}")
    full_md.append(f"- Faixa Atingimento Meta: {r.get('faixa_atingimento_meta','')}")
    full_md.append(f"- Faixa Categoria: {r.get('faixa_categoria','')}")
    full_md.append("")

    full_md.append(f"CPF/CNPJ - V")
    full_md.append(f"E-mail - {'V' if emails else 'X'} (tente trazer a chave PIX e-mail!)")
    full_md.append(f"Telefone - {'V' if telefones else 'X'}")
    full_md.append(f"Transacionou nos últimos 30d? {trans_30}\"")
    if faltantes:
        for p in faltantes:
            full_md.append(f"{p} X")
    else:
        full_md.append("✅ Tiene los productos básicos")
    if sow_line: full_md.append(sow_line)
    full_md.append("")
    if perdidos:
        for p in perdidos:
            full_md.append(f"- {p} (descadastrado)")
    else:
        full_md.append("- Ningún producto perdido identificado")
    full_md.append("")
    full_md.append("Status:")
    if data_atualizacao: full_md.append(f"- Fecha de actualización de cadastro: {data_atualizacao}")
    if pre_aprovado: full_md.append(f"- Pré-aprovado de cartões: {pre_aprovado}")
    full_md.append("")
    full_md.append(markdown_texto)  # Adicionar a descricao
    
    full = "\n".join(full_md)
    if len(full) > 10000:
        full = full[:9997] + "..."
    return {
        'titulo': titulo_local,
        'usuarioResponsavel': usuarioResponsavel,
        'usuarioSolicitante': usuarioSolicitante,
        'numeroAgencia': numeroAgencia,
        'descricao_md': full
    }

# Aplicar al dataframe filtrado
src_df = base_final if 'base_final' in globals() else (base_princ if 'base_princ' in globals() else (resultado_chamados if 'resultado_chamados' in globals() else None))
if src_df is None:
    raise RuntimeError("No se encontró base_final/base_princ/resultado_chamados en el entorno.")

# ---- Nuevo bloque: tratamiento / sanitización de columnas antes de aplicar la función ----
# Asegura tipos correctos y valores esperados para evitar errores (por ejemplo 'N' en lugar de 0)
def _to_bool_num(df, cols, map_vals=None, fillna=0):
    if map_vals is None:
        map_vals = {'S':1,'s':1,'Y':1,'y':1,'T':1,'t':1,'1':1,'0':0,'N':0,'n':0,'F':0,'f':0,'YES':1,'yes':1,'NO':0,'no':0}
    for c in cols:
        if c in df.columns:
            # reemplazar valores conocidos y forzar numérico
            df[c] = df[c].replace(map_vals)
            # quitar espacios y signos no numéricos para evitar conversion error
            df[c] = df[c].astype(str).str.strip()
            df[c] = df[c].replace({'': np.nan, 'nan': np.nan, 'None': np.nan})
            df[c] = pd.to_numeric(df[c], errors='coerce').fillna(fillna).astype(int)

def _to_numeric_clean(df, cols, remove_chars=None):
    for c in cols:
        if c in df.columns:
            s = df[c].astype(str).fillna('')
            if remove_chars:
                for ch in remove_chars:
                    s = s.str.replace(ch, '', regex=False)
            # normalizar coma decimal
            s = s.str.replace(',', '.', regex=False)
            df[c] = pd.to_numeric(s.replace({'': np.nan}), errors='coerce')

# columnas booleanas/flag que a veces vienen como 'S'/'N' o similares
bool_cols = ['mobi_transacionou_30d','cad_pix_ativo','debito_conta_ativo','possui_cartao_debito',
             'possui_cartao_credito','possui_folha_pagamento','possui_cobranca','possui_domicilio','possui_adquirencia','transacao_app']
_to_bool_num(src_df, bool_cols, fillna=0)

# columnas numéricas que pueden contener texto o '%'
_to_numeric_clean(src_df, ['soma_quedas','pontos_principalidade','var_pontos','mc_total_1','cash_total','isa_media','isa_media_3m_calc','mc_total_1','sow_cartao'], remove_chars=['%',' '])

# asegurar sow_cartao porcentaje como float si existía con '%'
if 'sow_cartao' in src_df.columns:
    # ya limpiado por _to_numeric_clean; queda como float o NaN

    # asegurar productos_basicos_faltantes como lista
    if 'produtos_basicos_faltantes' in src_df.columns:
        def _ensure_list(v):
            if isinstance(v, (list,tuple)):
                return v
            if pd.isna(v):
                return []
            try:
                # si viene como string con comas
                if isinstance(v,str) and ',' in v:
                    return [x.strip() for x in v.split(',') if x.strip()]
                return [v]
            except Exception:
                return []
        src_df['produtos_basicos_faltantes'] = src_df['produtos_basicos_faltantes'].apply(_ensure_list)

# normalizar telefone/email columnas (mantener campos comercial/celular/telefone sin renombrar)
for c in ['comercial','celular','telefone']:
    if c in src_df.columns:
        src_df[c] = src_df[c].astype(str).replace({'nan':None, 'None':None}).fillna('')

# normalizar cpf_cnpj llave
if 'cpf_cnpj' in src_df.columns:
    src_df['cpf_cnpj'] = src_df['cpf_cnpj'].astype(str).str.replace(r'\D','', regex=True).fillna('')
    src_df['cpf_cnpj'] = np.where(src_df['cpf_cnpj'].str.len() > 11,
                                  src_df['cpf_cnpj'].str.zfill(14),
                                  src_df['cpf_cnpj'].str.zfill(11))

# ---- Fin del bloque de tratamiento ----


outs = src_df.apply(lambda r: pd.Series(gerar_markdown_row(r)), axis=1)
src_df = src_df.assign(titulo=outs['titulo'].values, usuarioResponsavel=outs['usuarioResponsavel'].values,
                       usuarioSolicitante=outs['usuarioSolicitante'].values, numeroAgencia=outs['numeroAgencia'].values,
                       descricao=outs['descricao_md'].values)


if 'base_princ' in globals() and src_df is base_princ:
    base_princ['titulo'] = src_df['titulo']
    base_princ['usuarioResponsavel'] = src_df['usuarioResponsavel']
    base_princ['usuarioSolicitante'] = src_df['usuarioSolicitante']
    base_princ['descricao'] = src_df['descricao']

In [26]:
# Crear y guardar parquet final con todos los tratamientos aplicados
import os, numpy as np, pandas as pd
from IPython.display import display, Markdown

if 'base_princ' not in globals() or base_princ is None:
    raise RuntimeError("base_princ no está disponible. Ejecuta las celdas previas.")

df_final = base_princ.copy()

# Normalizar llaves/identificadores (regla zfill 11/14 para doc, 6 para conta, 2 para agencia, 3/6 para cartera)
df_final['cpf_cnpj'] = df_final['cpf_cnpj'].astype(str).str.replace(r'\D','', regex=True).fillna('')
df_final['cpf_cnpj'] = np.where(df_final['cpf_cnpj'].str.len() > 11,
                                df_final['cpf_cnpj'].str.zfill(14),
                                df_final['cpf_cnpj'].str.zfill(11))
if 'num_conta' in df_final.columns:
    df_final['num_conta'] = df_final['num_conta'].astype(str).fillna('').str.zfill(6)
if 'cod_agencia' in df_final.columns:
    df_final['cod_agencia'] = df_final['cod_agencia'].astype(str).fillna('').str.zfill(2)
if 'cod_carteira' in df_final.columns:
    df_final['cod_carteira'] = df_final['cod_carteira'].astype(str).fillna('')
    df_final['cod_carteira'] = np.where(df_final['cod_carteira'].str.len() > 3,
                                        df_final['cod_carteira'].str.zfill(6),
                                        df_final['cod_carteira'].str.zfill(3))

# Telefonos / emails: priorizar columnas comercial -> celular -> telefone (ya preservadas)
def _join_telef(row):
    nums = []
    for c in ('comercial','celular','telefone'):
        if c in row.index:
            v = row.get(c)
            if pd.notnull(v) and str(v).strip()!='':
                nums.append(str(v).strip())
    return ", ".join(dict.fromkeys(nums))  # dedup mantenido por orden

df_final['telefones'] = df_final.apply(_join_telef, axis=1)

# Emails: buscar columnas que contengan 'email' si no existe columna emails
if 'emails' not in df_final.columns:
    email_cols = [c for c in df_final.columns if re.search(r'email|e-mail', c, re.I)]
    if email_cols:
        df_final['emails'] = df_final[email_cols].apply(lambda r: ", ".join([str(x) for x in r.dropna().astype(str).unique()]) , axis=1)
    else:
        df_final['emails'] = np.nan

# Productos básicos faltantes: usar columna ya existente si existe, sino calcular list
prod_basic_cols = ['cad_pix_ativo','debito_conta_ativo','possui_cartao_debito','possui_cartao_credito','possui_folha_pagamento']
def _calc_faltantes(row):
    falt = []
    for c in prod_basic_cols:
        if c in row.index:
            v = row.get(c)
            # considerar 1/True como presente, 0/NaN/False como ausente
            try:
                if pd.isna(v) or float(v) == 0:
                    falt.append(c)
            except Exception:
                if not v:
                    falt.append(c)
    return falt

if 'produtos_basicos_faltantes' not in df_final.columns:
    df_final['produtos_basicos_faltantes'] = df_final.apply(_calc_faltantes, axis=1)
else:
    # asegurar tipo lista
    df_final['produtos_basicos_faltantes'] = df_final['produtos_basicos_faltantes'].apply(lambda v: v if isinstance(v, (list, tuple)) else ([] if pd.isna(v) or v=='' else [v]))

# Asegurar columnas ISA y crear isa_insight si falta
for c in ['isa_status','isa_media','isa_media_3m_calc','isa_insight']:
    if c not in df_final.columns:
        df_final[c] = np.nan

def _isa_insight_row(r):
    try:
        am = r.get('isa_media')
        am3 = r.get('isa_media_3m_calc')
        if pd.notnull(am) and pd.notnull(am3):
            amf = float(am)
            am3f = float(am3)
            if amf < am3f:
                return f"ISA en caída: actual {amf:.2f} < media 3M {am3f:.2f}"
            elif amf > am3f:
                return f"ISA mejoró: actual {amf:.2f} > media 3M {am3f:.2f}"
            else:
                return f"ISA estable: {amf:.2f}"
        if pd.notnull(am):
            return f"ISA actual: {float(am):.2f}"
        return ""
    except Exception:
        return ""

if df_final['isa_insight'].isnull().all():
    df_final['isa_insight'] = df_final.apply(_isa_insight_row, axis=1)

# Convertir listas a representaciones (opcional) para parquet: mantener listas para compatibilidad, además crear versión string para kanban
df_final['produtos_basicos_faltantes_str'] = df_final['produtos_basicos_faltantes'].apply(lambda x: ", ".join(x) if isinstance(x,(list,tuple)) and x else ("" if (pd.isna(x) or x==[]) else str(x)))

# Guardado atómico parquet final
out_final = os.path.join(RUTAS['saida_dir_giro'], 'base_princ_final.parquet')
tmp = out_final + '.tmp'
df_final.to_parquet(tmp, index=False)
os.replace(tmp, out_final)

display(Markdown(f"Parquet final guardado: {out_final} — registros: {len(df_final):,}"))

Parquet final guardado: C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\giro_de_carteira\base_princ_final.parquet — registros: 3,707

In [ ]:
# Crear y guardar parquet Kan‑ban con campos mínimos y estado/prioridad
import numpy as np, os, pandas as pd

if 'df_final' not in globals():
    # intentar cargar el parquet final si la variable no existe en memoria
    path_final = os.path.join(RUTAS['saida_dir_giro'], 'base_princ_final.parquet')
    if os.path.exists(path_final):
        df_final = pd.read_parquet(path_final)
    else:
        raise RuntimeError("No se encontró df_final en memoria ni el parquet final. Ejecuta la celda anterior.")

kan_cols = [
    'cpf_cnpj','num_conta','nom_associado','segmento','cod_agencia','cod_carteira',
    'soma_quedas','pontos_principalidade','var_pontos',
    'isa_media','isa_media_3m_calc','isa_insight',
    'produtos_basicos_faltantes_str','telefones','emails','status_associado',
    'soma_lost_produtos',  # Asegurar que la columna se incluye si existe
    'gestor','gestor_ldap','titulo','usuarioResponsavel','usuarioSolicitante','numeroAgencia','descricao'
]
# mantener las columnas que existan en df_final
kan_cols_existing = [c for c in kan_cols if c in df_final.columns]

kan = df_final[kan_cols_existing].copy()

# Calcular prioridad simple:
def _priority(r):
    try:
        sq = r.get('soma_quedas')
        isa = r.get('isa_media')
        isa3 = r.get('isa_media_3m_calc')
        # Alta: muchas caídas; Media: ISA por debajo de 3M o productos faltantes; Baja: else
        if pd.notnull(sq) and float(sq) >= 3:
            return 'Alta'
        if pd.notnull(isa) and pd.notnull(isa3) and float(isa) < float(isa3):
            return 'Media'
        # si tiene productos faltantes no vacía -> Media
        pb = r.get('produtos_basicos_faltantes_str','')
        if isinstance(pb,str) and pb.strip():
            return 'Media'
        return 'Baja'
    except Exception:
        return 'Baja'

kan['prioridade'] = kan.apply(_priority, axis=1)

# Estado Kanban inicial
kan['status_kanban'] = np.where(kan['produtos_basicos_faltantes_str'].fillna('')=='','Concluído','Pendiente')

# Orden de columnas recomendado para Kan‑ban
order = ['cpf_cnpj','num_conta','nom_associado','segmento','cod_agencia','cod_carteira',
         'soma_quedas','pontos_principalidade','var_pontos','percentual_atingimento','faixa_atingimento_meta','faixa_categoria',
         'isa_media','isa_media_3m_calc','isa_insight',
         'produtos_basicos_faltantes_str','telefones','emails','status_associado','gestor','gestor_ldap','prioridade','status_kanban']
kan = kan[[c for c in order if c in kan.columns]]

# Guardado atómico parquet kanban
out_kan = os.path.join(RUTAS['saida_dir_giro'], 'giro_cards_kanban.parquet')
tmpk = out_kan + '.tmp'
kan.to_parquet(tmpk, index=False)
os.replace(tmpk, out_kan)

display(Markdown(f"Parquet Kan‑ban guardado: {out_kan} — registros: {len(kan):,}"))

Parquet Kan‑ban guardado: C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\giro_de_carteira\giro_cards_kanban.parquet — registros: 3,707

In [ ]:
# Teste com 4 CPFs fornecidos pelo usuario, obtendo dados do parquet kanban ou final
from IPython.display import Markdown, display
import pandas as pd, os
import numpy as np

# Variáveis para os CPFs (preencher com os valores reais)
cpf1 = '02032337940'  # Preencher CPF 1
cpf2 = '01188254960'  # Preencher CPF 2
cpf3 = '00672137000192'  # Preencher CPF 3
cpf4 = '06295052924'  # Preencher CPF 4

# Lista de CPFs para teste
cpfs_teste = [cpf1, cpf2, cpf3, cpf4]

# Carregar dados do parquet kanban ou final (priorizar kanban se existir)
RUTAS = globals().get('RUTAS', {})
base_dir = RUTAS.get('saida_dir_giro', '')
path_kanban = os.path.join(base_dir, 'giro_cards_kanban.parquet')
path_final = os.path.join(base_dir, 'associados_queda_principalidade.parquet')

df_teste = None
if os.path.exists(path_kanban):
    df_teste = pd.read_parquet(path_kanban)
elif os.path.exists(path_final):
    df_teste = pd.read_parquet(path_final)
else:
    raise FileNotFoundError("Nenhum parquet encontrado para teste.")

if df_teste is None or df_teste.empty:
    raise RuntimeError("DataFrame vazio ou não encontrado.")

# Normalizar CPFs para busca
df_teste['cpf_cnpj'] = (
    df_teste['cpf_cnpj']
    .astype(str)
    .str.replace(r'\D', '', regex=True)
    .fillna('')
)
df_teste['cpf_cnpj'] = np.where(
    df_teste['cpf_cnpj'].str.len() > 11,
    df_teste['cpf_cnpj'].str.zfill(14),
    df_teste['cpf_cnpj'].str.zfill(11)
)

# Para cada CPF, buscar no DataFrame e gerar markdown
for cpf in cpfs_teste:
    # Selecionar apenas o CPF atual
    sel = df_teste.loc[df_teste['cpf_cnpj'] == cpf]
    if not sel.empty:
        r = sel.iloc[0]

        # Build descricao_dict and nome
        prod_cols_map = {
            'Pix': 'cad_pix_ativo',
            'Débito automático': 'debito_conta_ativo',
            'Conta salário': 'possui_folha_pagamento',
            'Cartão de débito': 'possui_cartao_debito',
            'Cartão de crédito': 'possui_cartao_credito',
            'Canais digitais': 'transacao_app',
            'Open Finance (receptor)': 'ativou_open_finance',
            'Domicílio': 'possui_domicilio',
            'Credenciamento ativo': 'possui_adquirencia',
            'Cobrança': 'possui_cobranca',
            'Folha de pagamento': 'possui_folha_pagamento'
        }
        prod_cols_map = {k: v for k, v in prod_cols_map.items() if v in df_teste.columns}

        fluxo_map = {
            'Valor movimentado': 'cash_total',
            'Cartão (SOW)': 'sow_cartao',
            'Cash-in': 'cash_in',
            'Cash-out': 'cash_out'
        }
        fluxo_map = {k: v for k, v in fluxo_map.items() if v in df_teste.columns}

        def _fmt(v):
            try:
                if pd.isna(v):
                    return ''
                return f"{float(v):,.2f}"
            except Exception:
                return str(v)

        acesso = {
            label: (bool(float(r.get(col, 0)) != 0) if col in r.index else False)
            for label, col in prod_cols_map.items()
        }
        fluxo = {}
        for label, col in fluxo_map.items():
            if col in r.index and pd.notnull(r.get(col)):
                fluxo[label] = _fmt(r.get(col))

        descricao_dict = {"Acesso": acesso, "Fluxo de Caixa": fluxo}

        nome = r.get('nom_associado') or ''

        # Partial markdown
        def gerar_markdown(d, nome_associado):
            md = ["## DADOS ASSOCIADO"]
            md.append(f"- Nome Associado: {nome_associado}")
            md.append(f"- CPF/CNPJ: {r.get('cpf_cnpj', '')}")
            md.append(f"- Agencia: {r.get('cod_agencia', '')}")
            if 'num_conta' in r.index and pd.notnull(r.get('num_conta')):
                md.append(f"- Conta: {r.get('num_conta')}")
            md.append("")
            md.append("## Grupos de Soluciones")
            md.append("")
            md.append("### 🔑 Acceso")
            for k, v in d["Acesso"].items():
                simbolo = "✅" if v else "❌"
                md.append(f"- {simbolo} {k}")
            md.append("")
            md.append("### 💰 Fluxo de Caixa")
            if d["Fluxo de Caixa"]:
                for k, v in d["Fluxo de Caixa"].items():
                    md.append(f"- **{k}:** {v}")
            else:
                md.append("- Sem métricas de fluxo disponiveis")
            return "\n".join(md)

        markdown_partial = gerar_markdown(descricao_dict, nome)

        # Full markdown (assume função gerar_markdown_row já definida em outra célula)
        md_dict = gerar_markdown_row(r)
        markdown_full = md_dict['descricao_md']

        # Combine
        combined_md = f"{markdown_partial}\n\n---\n\n{markdown_full}"
        display(Markdown(f"### Teste para CPF: {cpf}\n{combined_md}"))
    else:
        display(Markdown(f"### CPF {cpf} não encontrado no parquet."))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [19]:
# Generación de Reporte de Insights
from IPython.display import display, Markdown
from collections import Counter
import pandas as pd
import os

# Cargar el dataframe final (preferiblemente el de Kanban por tener la prioridad)
report_path = os.path.join(RUTAS['saida_dir_giro'], 'giro_cards_kanban.parquet')
if not os.path.exists(report_path):
    report_path = os.path.join(RUTAS['saida_dir_giro'], 'base_princ_final.parquet')

if not os.path.exists(report_path):
    raise FileNotFoundError("No se encontró el archivo parquet final para generar el reporte.")

df_report = pd.read_parquet(report_path)

# --- Generación de Insights ---

# 1. Cantidad total de associados para contactar
total_contactos = len(df_report)

# 2. Distribución por segmento
dist_segmento = df_report['segmento'].value_counts()

# 3. Distribución por prioridad (si existe)
dist_prioridad = None
if 'prioridade' in df_report.columns:
    dist_prioridad = df_report['prioridade'].value_counts()

# 4. Productos básicos faltantes más comunes
top_5_faltantes = []
if 'produtos_basicos_faltantes_str' in df_report.columns:
    all_faltantes = []
    df_report['produtos_basicos_faltantes_str'].dropna().apply(lambda x: all_faltantes.extend([p.strip() for p in x.split(',') if p.strip()]))
    top_5_faltantes = Counter(all_faltantes).most_common(5)

# 5. Distribución por Faixa Categoria
dist_faixa_categoria = None
if 'faixa_categoria' in df_report.columns:
    dist_faixa_categoria = df_report['faixa_categoria'].value_counts()

# --- Construcción del Reporte Markdown ---
report_md = [f"# Reporte de Insights: Caída de Principalidade ({date.today().strftime('%Y-%m-%d')})"]
report_md.append("\nEste reporte resume los principales insights obtenidos del análisis de associados con caída recurrente en el índice de principalidade.")

report_md.append(f"\n## 1. Total de Oportunidades")
report_md.append(f"- **Total de associados para contactar:** {total_contactos}")

report_md.append("\n## 2. Distribución de Oportunidades")

# Segmento
report_md.append("### Por Segmento")
for seg, count in dist_segmento.items():
    report_md.append(f"- **{seg}:** {count} associados ({count/total_contactos:.1%})")

# Prioridad
if dist_prioridad is not None:
    report_md.append("\n### Por Prioridad de Contacto")
    for prio, count in dist_prioridad.items():
        report_md.append(f"- **{prio}:** {count} associados ({count/total_contactos:.1%})")

# Faixa Categoria
if dist_faixa_categoria is not None:
    report_md.append("\n### Por Faixa de Categoría")
    for faixa, count in dist_faixa_categoria.items():
        report_md.append(f"- **{faixa}:** {count} associados")

report_md.append("\n## 3. Principales Productos Faltantes")
report_md.append("Top 5 productos con mayor oportunidad de contratación entre los associados identificados:")
if top_5_faltantes:
    for i, (prod, count) in enumerate(top_5_faltantes):
        report_md.append(f"{i+1}. **{prod.replace('_', ' ').title()}**: {count} veces")
else:
    report_md.append("- No se encontró información de productos faltantes.")

# --- Sección de Mayores Caídas ---
report_md.append("\n## 4. Análisis de Mayores Caídas por Segmento")

# Calcular caída de ISA
if 'isa_media' in df_report.columns and 'isa_media_3m_calc' in df_report.columns:
    df_report['isa_drop'] = df_report['isa_media_3m_calc'] - df_report['isa_media']

for segmento in df_report['segmento'].unique():
    report_md.append(f"\n### Segmento: {segmento}")
    df_seg = df_report[df_report['segmento'] == segmento]

    # a. Mayor caída de Principalidade (var_pontos es negativo)
    report_md.append("\n**Top 3 - Mayor Caída de Principalidade:**")
    if 'var_pontos' in df_seg.columns and not df_seg['var_pontos'].isnull().all():
        top_princ = df_seg.sort_values('var_pontos', ascending=True).head(3)
        for _, r in top_princ.iterrows():
            report_md.append(f"- {r.get('nom_associado','N/A')} (CPF/CNPJ: ...{r.get('cpf_cnpj','')[-4:]}): **{r.get('var_pontos'):.0f} puntos**")
    else:
        report_md.append("- Datos no disponibles.")

    # b. Mayor caída de ISA
    report_md.append("\n**Top 3 - Mayor Caída de ISA:**")
    if 'isa_drop' in df_seg.columns and not df_seg['isa_drop'].isnull().all():
        top_isa = df_seg[df_seg['isa_drop'] > 0].sort_values('isa_drop', ascending=False).head(3)
        if not top_isa.empty:
            for _, r in top_isa.iterrows():
                report_md.append(f"- {r.get('nom_associado','N/A')} (CPF/CNPJ: ...{r.get('cpf_cnpj','')[-4:]}): **caída de {r.get('isa_drop'):.2f}** (de {r.get('isa_media_3m_calc'):.2f} a {r.get('isa_media'):.2f})")
        else:
            report_md.append("- No se encontraron caídas de ISA en este segmento.")
    else:
        report_md.append("- Datos no disponibles.")

    # c. Mayor pérdida de productos
    report_md.append("\n**Top 3 - Mayor Pérdida de Productos:**")
    if 'soma_lost_produtos' in df_seg.columns and not df_seg['soma_lost_produtos'].isnull().all():
        top_lost = df_seg[df_seg['soma_lost_produtos'] > 0].sort_values('soma_lost_produtos', ascending=False).head(3)
        if not top_lost.empty:
            for _, r in top_lost.iterrows():
                report_md.append(f"- {r.get('nom_associado','N/A')} (CPF/CNPJ: ...{r.get('cpf_cnpj','')[-4:]}): **{r.get('soma_lost_produtos'):.0f} productos perdidos**")
        else:
            report_md.append("- No se identificaron pérdidas de productos en este segmento.")
    else:
        report_md.append("- Datos no disponibles.")


# --- Guardar y Mostrar Reporte ---
final_report_str = "\n".join(report_md)
report_file_path = os.path.join(RUTAS['saida_dir_giro'], 'reporte_insights.md')

with open(report_file_path, 'w', encoding='utf-8') as f:
    f.write(final_report_str)

display(Markdown(final_report_str))
print(f"\nReporte guardado en: {report_file_path}")

# Reporte de Insights: Caída de Principalidade (2025-08-27)

Este reporte resume los principales insights obtenidos del análisis de associados con caída recurrente en el índice de principalidade.

## 1. Total de Oportunidades
- **Total de associados para contactar:** 3707

## 2. Distribución de Oportunidades
### Por Segmento
- **PF:** 2348 associados (63.3%)
- **PJ:** 963 associados (26.0%)
- **AGRO:** 396 associados (10.7%)

### Por Prioridad de Contacto
- **Alta:** 3707 associados (100.0%)

## 3. Principales Productos Faltantes
Top 5 productos con mayor oportunidad de contratación entre los associados identificados:
1. **Possui Folha Pagamento**: 3645 veces
2. **Possui Adquirencia**: 3586 veces
3. **Ativou Open Finance**: 3554 veces
4. **Possui Domicilio**: 3500 veces
5. **Possui Cobranca**: 3477 veces

## 4. Análisis de Mayores Caídas por Segmento

### Segmento: PF

**Top 3 - Mayor Caída de Principalidade:**
- N/A (CPF/CNPJ: ...8924): **-99 puntos**
- N/A (CPF/CNPJ: ...4985): **-94 puntos**
- N/A (CPF/CNPJ: ...2957): **-89 puntos**

**Top 3 - Mayor Caída de ISA:**
- Datos no disponibles.

**Top 3 - Mayor Pérdida de Productos:**
- Datos no disponibles.

### Segmento: PJ

**Top 3 - Mayor Caída de Principalidade:**
- N/A (CPF/CNPJ: ...0185): **-74 puntos**
- N/A (CPF/CNPJ: ...0117): **-66 puntos**
- N/A (CPF/CNPJ: ...0141): **-65 puntos**

**Top 3 - Mayor Caída de ISA:**
- Datos no disponibles.

**Top 3 - Mayor Pérdida de Productos:**
- Datos no disponibles.

### Segmento: AGRO

**Top 3 - Mayor Caída de Principalidade:**
- N/A (CPF/CNPJ: ...9804): **-95 puntos**
- N/A (CPF/CNPJ: ...1904): **-71 puntos**
- N/A (CPF/CNPJ: ...9949): **-65 puntos**

**Top 3 - Mayor Caída de ISA:**
- Datos no disponibles.

**Top 3 - Mayor Pérdida de Productos:**
- Datos no disponibles.


Reporte guardado en: C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\giro_de_carteira\reporte_insights.md
